In [42]:
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset , DataLoader
import numpy as np
import torch.nn as nn
import os
import torch

In [43]:
folder_path = "./Dataset_png"
target_list={}
data={}
to_tensor = transforms.ToTensor()
for number,file_name in enumerate( os.listdir(folder_path)):
    file_path=os.path.join(folder_path,file_name)
    image=Image.open(file_path)
    image_tensor=to_tensor(image)
    if file_name.startswith("front"):
        target_list[number]="front"
    elif  file_name.startswith("back"):
        target_list[number]="back"
    data[number]=image_tensor

In [44]:
imgs = torch.stack(list(data.values()), dim=3)
imgs.shape

torch.Size([1, 288, 432, 202])

In [45]:
imgs.view(1, -1).mean(dim=1) 

tensor([0.0547])

In [46]:
imgs.view(1, -1).std(dim=1)

tensor([0.1238])

In [47]:
to_tensor = transforms.Compose([transforms.Resize([288,288]),
                                transforms.ToTensor(),
                                transforms.Normalize((0.0547), #各通道的平均值
                                                     (0.1238))])

for number,file_name in enumerate( os.listdir(folder_path)):
    file_path=os.path.join(folder_path,file_name)
    image=Image.open(file_path)
    image_tensor=to_tensor(image)
    if file_name.startswith("front"):
        target_list[number]=0
    elif  file_name.startswith("back"):
        target_list[number]=1
    data[number]=image_tensor

In [48]:
def data_loader(file_path):
    image=Image.open(file_path)
    image_tensor=to_tensor(image)
    return image_tensor
#照片的所有路徑
images_path=[]
for filename in os.listdir(folder_path):
    file_path=os.path.join(folder_path,filename)
    images_path.append(file_path)

In [49]:
#target換成tensor
data_target_t=torch.tensor(list(target_list.values()))
data_target_t.shape

torch.Size([202])

In [50]:
#建立自己的資料集
class TrainSet(Dataset):
    def __init__(self,loader=data_loader):
        self.images=images_path
        self.target=data_target_t
        self.loader=loader
    def __getitem__(self,index):
        fn=self.images[index]
        img = self.loader(fn)
        target=self.target[index]
        return img , target
    def __len__(self):
        return len(self.images)

In [51]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1) 
        self.act1 = nn.Tanh()
        self.pool1 = nn.MaxPool2d(2) 
        self.conv2 = nn.Conv2d(16, 8, kernel_size=3, padding=1) 
        self.act2 = nn.Tanh()
        self.pool2 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(8 * 72 * 72, 32) 
        self.act3 = nn.Tanh()
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        out = self.pool1(self.act1(self.conv1(x)))
        out = self.pool2(self.act2(self.conv2(out)))
        out = out.view(-1, 8 * 72 * 72)  #之前省略的部分（扁平化卷積模組的輸出）
        out = self.act3(self.fc1(out))
        out = self.fc2(out)
        return out

In [52]:
import datetime 
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):  #控制訓練週期的迴圈，注意範圍是從1開始到n_epochs，而非從0開始
        loss_train = 0.0
        for imgs, labels in train_loader:  #走訪train_loader，每次會取出一批次的訓練資料及標籤
            outputs = model(imgs)  #將一批次資料餵入模型中
            loss = loss_fn(outputs, labels)  #計算損失（最小化的目標）
            optimizer.zero_grad()  #將上一輪的梯度清除
            loss.backward()  #反向運行一次，以便取得損失對所有可訓練參數的梯度 
            optimizer.step()  #更新模型參數
            loss_train += loss.item()  #將此次訓練週期中的所有損失加總起來；請記得，必須用item()將損失轉換為Python中的數字
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))  #將損失總和除以批次量（即train_loader的長度），以取得每批次的平均損失（相較於總和，以平均值來測量損失更加直觀）

In [53]:
model = Net()#建立一個Net()物件
train_data = TrainSet()
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)  #先將cifar資料集隨機洗牌，然後我們每次讀取時會由其中抽取一批次（64筆）的資料
model = Net()  #建立模型
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)  #隨機梯度下降優化器
loss_fn = nn.CrossEntropyLoss()  #第7.10節中提過的交叉熵損失函數

training_loop(  #呼叫稍早之前定義的訓練迴圈
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2023-12-17 18:37:55.910053 Epoch 1, Training loss 0.7730515748262405
2023-12-17 18:38:14.850264 Epoch 10, Training loss 0.4884546399116516
2023-12-17 18:38:35.833419 Epoch 20, Training loss 0.32774411886930466
2023-12-17 18:38:56.889150 Epoch 30, Training loss 0.1525781136006117
2023-12-17 18:39:18.034051 Epoch 40, Training loss 0.10002916678786278
2023-12-17 18:39:39.023143 Epoch 50, Training loss 0.06670167483389378
2023-12-17 18:40:00.588076 Epoch 60, Training loss 0.04897028859704733
2023-12-17 18:40:22.194997 Epoch 70, Training loss 0.04139694198966026
2023-12-17 18:40:43.466262 Epoch 80, Training loss 0.031691205222159624
2023-12-17 18:41:04.500306 Epoch 90, Training loss 0.026067625731229782
2023-12-17 18:41:25.842527 Epoch 100, Training loss 0.055695123039186


In [64]:
imgs=data_loader("./Valid_png/front2.png")
imgs=imgs.unsqueeze(0)
outputs=model(imgs)
_, predicted = torch.max(outputs, dim=1) #取得最大值所在的索引，並存進predicted陣列
outputs,predicted

(tensor([[-2.5552,  2.5873]], grad_fn=<AddmmBackward0>), tensor([1]))